In [ ]:
# Hidden Markov Models for Bitcoin Regime Detection Using pomegranate and Alpha-Stable Distributions

# This notebook will guide you through detecting regime changes in simulated Bitcoin data using Hidden Markov Models (HMM)
# with non-Gaussian Alpha-Stable distributions, leveraging the pomegranate library.

# In financial markets, regime changes indicate shifts between different market conditions, such as bullish, bearish,
# and sideways markets. These changes often involve varying volatility conditions.

# 1. First, we will simulate data that mimics Bitcoin returns under various regimes.
# 2. Then, we will fit an HMM model using the Alpha-Stable distribution to account for Bitcoin's non-Gaussian return distribution.
# 3. Finally, we will predict the market regimes, visualize them, and export the results for further analysis.

# Let's get started!

# ## Step 1: Simulate Bitcoin-like Data for Regimes

# We will simulate different market regimes, including bull markets, bear markets, and range markets, each with low and high volatility.


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Set random seed for reproducibility
np.random.seed(42)

# Define the number of samples per regime
n_samples = 1000

# Simulate returns for different regimes
regimes = {
    'bull_low_vol': np.random.normal(0.001, 0.01, size=n_samples),   # Bull market, low volatility
    'bull_high_vol': np.random.normal(0.001, 0.05, size=n_samples),  # Bull market, high volatility
    'bear_low_vol': np.random.normal(-0.001, 0.01, size=n_samples),  # Bear market, low volatility
    'bear_high_vol': np.random.normal(-0.001, 0.05, size=n_samples), # Bear market, high volatility
    'range_low_vol': np.random.normal(0, 0.01, size=n_samples),      # Range market, low volatility
    'range_high_vol': np.random.normal(0, 0.05, size=n_samples)      # Range market, high volatility
}

# Combine regimes into a single time series by concatenating the different arrays
data = np.concatenate([regimes['bull_low_vol'], regimes['bull_high_vol'],
                       regimes['bear_low_vol'], regimes['bear_high_vol'],
                       regimes['range_low_vol'], regimes['range_high_vol']])

# Create a date range for the simulated data
dates = pd.date_range(start='2023-01-01', periods=len(data))

# Create a DataFrame to hold the simulated data
df = pd.DataFrame({'Date': dates, 'Returns': data})
df.set_index('Date', inplace=True)

# Plot the simulated returns
df['Returns'].plot(title='Simulated Bitcoin-like Returns', figsize=(10, 6))
plt.show()

In [ ]:
# ### Explanation:
# In this step, we have simulated 6 different regimes:
# - Bull market with low volatility
# - Bull market with high volatility
# - Bear market with low volatility
# - Bear market with high volatility
# - Range (sideways) market with low volatility
# - Range market with high volatility

# These regimes are concatenated into a single time series, and we visualize the combined returns.

# ## Step 2: Fit the Hidden Markov Model with Alpha-Stable Distribution

# Now that we have our simulated returns, we will use the pomegranate library to fit a Hidden Markov Model (HMM).
# We will assume the returns are generated from an Alpha-Stable distribution, which is more flexible than the traditional
# Gaussian distribution and can model Bitcoin's fat tails and volatility clustering.


In [ ]:
from pomegranate import HiddenMarkovModel, AlphaStableDistribution

# Define the number of hidden states (regimes) - 6 regimes: bull/bear/range with low/high volatility
n_components = 6

# Initialize Alpha-Stable distributions for each state (regime)
states = []
for i in range(n_components):
    dist = AlphaStableDistribution(alpha=1.7, beta=0, gamma=0.02, delta=0.001)  # Example parameters for Alpha-Stable
    states.append(dist)

# Create and fit the Hidden Markov Model
model = HiddenMarkovModel.from_samples(AlphaStableDistribution, n_components=n_components,
                                       X=df['Returns'].values.reshape(-1, 1),
                                       algorithm='baum-welch', n_iter=100)

# Print model summary
print(model)

In [ ]:
# ### Explanation:
# - We create an HMM with 6 hidden states, each corresponding to one of the regimes (bull/bear/range with low/high volatility).
# - For each hidden state, we initialize an Alpha-Stable distribution with specific parameters (alpha, beta, gamma, delta).
#   These parameters allow the distribution to model fat tails and asymmetry commonly seen in financial data.
# - The model is trained using the Baum-Welch algorithm, which adjusts the transition probabilities and state distributions
#   to best fit the observed data.

# ## Step 3: Predict the Market Regimes

# After fitting the HMM, we can predict the hidden regimes (market conditions) that correspond to each time step in our data.

# Predict the hidden states (regimes) for the entire dataset

In [ ]:
predicted_states = model.predict(df['Returns'].values.reshape(-1, 1))

# Add the predicted regimes to the DataFrame
df['Regime'] = predicted_states

# Plot the returns with the predicted regimes
colors = {0: 'green', 1: 'blue', 2: 'red', 3: 'orange', 4: 'purple', 5: 'yellow'}
plt.figure(figsize=(10,6))
for state in range(n_components):
    state_data = df[df['Regime'] == state]
    plt.scatter(state_data.index, state_data['Returns'], color=colors[state], label=f'State {state}', s=1)
plt.legend()
plt.title('Simulated Bitcoin Regimes Detected by HMM')
plt.show()

In [ ]:
# ### Explanation:
# - The `model.predict()` method assigns each data point (return value) to a hidden state (regime).
# - We visualize the predicted regimes by coloring each time period based on the regime assigned by the HMM.
# - Each state (0 to 5) corresponds to one of the 6 simulated regimes.

# ## Step 4: Export the Results to CSV

# Export the DataFrame with the predicted regimes to a CSV file for further analysis.


In [ ]:
df.to_csv('regimes_prediction.csv')

In [ ]:
# ### Explanation:
# - The DataFrame now contains the simulated returns and the corresponding regime (market condition) at each time step.
# - We export the results to a CSV file to facilitate further analysis, such as backtesting or performance evaluation of trading strategies based on regime detection.

# ## Step 5: Further Analysis

# In this notebook, we used simulated data to demonstrate how to detect regime changes using a Hidden Markov Model with Alpha-Stable distributions.
# The pomegranate library provides flexibility in modeling the non-Gaussian nature of financial returns, making it a good choice for regime detection in markets like Bitcoin.

# You can extend this analysis by:
# - Using real Bitcoin data (e.g., from yfinance or Binance API) instead of simulated data.
# - Tuning the parameters of the Alpha-Stable distribution to better match the characteristics of Bitcoin returns.
# - Incorporating additional features, such as volatility or volume, to improve regime detection accuracy.